#### Extract Wildfire Events from Publicly available Wildfire Datasets
This notebook extracts wildfire events from two publicly available sources of wildfire datasets:
1. [Monitoring Trends in Burn Severity (MTBS)](https://www.mtbs.gov) over the United States of America
2. [Sentinel-2 Wildfire Change Detection](https://ieee-dataport.org/documents/sentinel-2-wildfire-change-detection-s2-wcd) across Europe and Australia

In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr

from shapely.geometry import mapping
from shapely.wkt import loads
from shapely.geometry import shape

import warnings
warnings.filterwarnings("ignore")

from src.utils import extract

Load S2WCD Wildfire Events

In [3]:
s2_wcd_fire_df = pd.read_csv("data/s2_wcd_fires.csv")
s2_wcd_fire_gpd = gpd.GeoDataFrame(s2_wcd_fire_df.copy())
s2_wcd_fire_gpd.set_geometry(s2_wcd_fire_gpd["geometry"].apply(loads), inplace=True)
s2_wcd_fire_gpd.set_crs("EPSG:4326", inplace=True)
s2_wcd_fire_gpd = extract.get_fire_path(s2_wcd_fire_df)
s2_wcd_fire_gpd["source"] = "S2 WCD"
s2_wcd_fire_gpd.head()

,location,geometry,pre_date,post_date,mask_path,source
0,Almonaster,"MULTIPOLYGON (((-1.94651 41.37228, -1.94652 41...",2022-07-12,2022-07-27,/workspace/Rufai/data/S2-WCD/Almonaster/cm/cm.tif,S2 WCD
1,Attica,"MULTIPOLYGON (((23.2936 38.09311, 23.2936 38.0...",2021-08-16,2021-08-28,/workspace/Rufai/data/S2-WCD/Attica/cm/cm.tif,S2 WCD
2,Australia_1,"POLYGON ((116.09595 -31.70046, 116.09595 -31.7...",2021-01-31,2021-02-20,/workspace/Rufai/data/S2-WCD/Australia_1/cm/cm...,S2 WCD
3,Australia_2,"MULTIPOLYGON (((116.0803 -31.75746, 116.08009 ...",2021-01-31,2021-02-20,/workspace/Rufai/data/S2-WCD/Australia_2/cm/cm...,S2 WCD
4,Bejis,"MULTIPOLYGON (((-0.71135 39.88491, -0.71147 39...",2022-08-08,2022-08-23,/workspace/Rufai/data/S2-WCD/Bejis/cm/cm.tif,S2 WCD


Load MTBS Wildfire Events

In [4]:
mtbs_gdf = gpd.read_file("data/mtbs/mtbs_perims_DD.shp")
mtbs_gdf["start_date"] = mtbs_gdf["Event_ID"].apply(extract.add_start_date)
mtbs_gdf = gpd.read_file("data/mtbs/mtbs_perims_DD.shp")
mtbs_gdf_filtered = extract.filter_fires_by_date(mtbs_gdf, "20210101", "20240101")
mtbs_gdf_filtered.head()

,Event_ID,irwinID,Incid_Name,Incid_Type,Map_ID,Map_Prog,Asmnt_Type,BurnBndAc,BurnBndLat,BurnBndLon,...,Low_T,Mod_T,High_T,Comment,geometry,start_date,location,source,pre_date,post_date
17161,NJ3973407472120220619,400501E0-C028-41BA-BF7E-189BFAA0F876,MULLICA RIVER FIRE,Wildfire,10024293,MTBS,Initial,13082,39.689,-74.681,...,30,306,600,None,"POLYGON ((-74.69252 39.68403, -74.6923 39.6837...",2022-06-19,MULLICA RIVER FIRE,MTBS,2021-06-07,2022-07-04
17163,NC3565607641720220619,FF845212-7195-428F-88B5-1E904EA26003,FEREBEE ROAD,Wildfire,10024302,MTBS,Initial,1954,35.673,-76.427,...,50,9999,9999,None,"POLYGON ((-76.42425 35.65565, -76.42447 35.654...",2022-06-19,FEREBEE ROAD,MTBS,2021-07-25,2022-07-20
17165,FL2576508042220220329,CFBFE97D-0AAB-4777-89DB-6C5AE68BA263,137 AVE,Wildfire,10024307,MTBS,Initial,857,25.769,-80.433,...,50,450,9999,None,"POLYGON ((-80.43589 25.77237, -80.43507 25.772...",2022-03-29,137 AVE,MTBS,2022-03-26,2022-04-10
17166,FL2632608099820220328,07627D47-33EE-40C9-8773-DF8DDCF4D748,INTERCEPTOR,Wildfire,10024309,MTBS,Initial,4143,26.297,-81.059,...,50,9999,9999,None,"POLYGON ((-81.08343 26.29129, -81.08347 26.291...",2022-03-28,INTERCEPTOR,MTBS,2022-03-26,2022-04-10
17168,FL2540408046920220426,C8F015EF-F5A2-402A-BF64-6D89D8A7ECAB,125 MILE MARKER,Wildfire,10024312,MTBS,Initial,830,25.411,-80.479,...,50,9999,9999,None,"POLYGON ((-80.48006 25.41823, -80.48017 25.417...",2022-04-26,125 MILE MARKER,MTBS,2020-05-05,2022-05-25


In [ ]:
all_fire_events = s2_wcd_fire_gpd.merge(
    mtbs_gdf_filtered, 
    on=["geometry", "pre_date", "post_date", 'location', 'source'],
    how="outer"
)
all_fire_events = all_fire_events[[c for c in 
                       ['source',
                        'location',
                        "geometry",
                        "pre_date",
                        "post_date",
                        ] if c in all_fire_events.columns]]
all_fire_events.head()

,source,location,geometry,pre_date,post_date
0,MTBS,DEER CREEK 2,"POLYGON ((-105.98965 44.51629, -105.99044 44.5...",2021-07-12,2021-08-13
1,MTBS,TA-43,"POLYGON ((-104.91643 38.46597, -104.91643 38.4...",2022-04-18,2022-05-03
2,MTBS,POLO,"POLYGON ((-97.74296 26.82487, -97.74412 26.826...",2022-01-29,2022-02-14
3,MTBS,OHSP,"POLYGON ((-96.15484 36.73975, -96.15416 36.738...",2021-02-23,2021-03-15
4,MTBS,TILTILL,"POLYGON ((-119.6892 37.99176, -119.6893 37.991...",2021-07-13,2022-07-16
...,...,...,...,...,...
781,S2 WCD,Almonaster,"MULTIPOLYGON (((-1.94651 41.37228, -1.94652 41...",2022-07-12,2022-07-27
782,S2 WCD,Monaco,"MULTIPOLYGON (((6.34477 43.33507, 6.34476 43.3...",2021-08-02,2021-08-27
783,S2 WCD,Rhodes_3,"MULTIPOLYGON (((27.97081 36.16503, 27.9708 36....",2023-07-13,2023-08-17
784,S2 WCD,Navvara,"MULTIPOLYGON (((-1.57204 42.51661, -1.57216 42...",2022-05-28,2022-07-07


In [6]:
print("Number of fire events:", len(all_fire_events))

Number of fire events: 786


In [10]:
all_fire_events.to_csv("data/fire_events_aoi.csv")